In [ ]:
#Import the necessary libraries
import os
import cv2
import numpy as np
import time

In [ ]:
#Import Directory for OpenCV screenshots
screenshot_dir = 'CV_Screenshots'
os.makedirs(screenshot_dir, exist_ok=True)

In [ ]:
#PHASE 1: CAMERA ACCESS AND SETUP

def find_working_camera():
    for i in range(5):
        print(f"   Testing camera {i}...", end=" ")
        
        cap = cv2.VideoCapture(i, cv2.CAP_AVFOUNDATION)
        
        if cap.isOpened():
            # Try to read a frame
            ret, frame = cap.read()
            if ret and frame is not None:
                h, w = frame.shape[:2]
                print(f"✅ WORKS! ({w}x{h})")
                cap.release()
                return i
            else:
                print("❌ Opens but can't grab frames")
        else:
            print("❌ Can't open")
        cap.release()
    return None


def test_camera_access():
    camera_index = find_working_camera()
    
    if camera_index is None:
        print("\n❌ ERROR: No working camera found!")
        return
    
    print(f"\n✅ Using camera {camera_index}")
    
    # Open camera with AVFoundation backend (better for Mac)
    cap = cv2.VideoCapture(camera_index, cv2.CAP_AVFOUNDATION)
    
    # Force better settings
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    cap.set(cv2.CAP_PROP_FPS, 30)
    
    print("✅ Camera opened successfully!")
    print("\nCamera window should open now...\n")
    
    # Get camera properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    print(f"📊 Camera Info:")
    print(f"   Resolution: {width}x{height}")
    print(f"   FPS: {fps}")
    
    screenshot_count = 0
    frame_count = 0
    start_time = time.time()
    
    # Main camera loop
    while True:
        # Read frame from camera
        ret, frame = cap.read()
        
        if not ret:
            print("❌ Failed to grab frame")
            break
        
        frame_count += 1
        
        # Calculate actual FPS
        elapsed_time = time.time() - start_time
        if elapsed_time > 0:
            actual_fps = frame_count / elapsed_time
        else:
            actual_fps = 0
        
        # Add FPS counter to frame
        cv2.putText(
            frame, 
            f"FPS: {actual_fps:.1f}", 
            (10, 30),
            cv2.FONT_HERSHEY_SIMPLEX, 
            0.7, 
            (0, 255, 0), 
            2
        )
        
        # Add instructions
        cv2.putText(
            frame, 
            "Press 'q' to quit, 's' for screenshot", 
            (10, height - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 
            0.5, 
            (255, 255, 255), 
            1
        )
        
        # Display the frame
        cv2.imshow('Friend Classifier - Phase 1', frame)
        
        # Handle keyboard input
        key = cv2.waitKey(1) & 0xFF
        
        if key == ord('q') or key == 27:  # 'q' or ESC
            print("\n👋 Quitting...")
            break
        elif key == ord('s'):  # Save screenshot
            screenshot_count += 1
            filename = os.path.join(screenshot_dir, f'screenshot_{screenshot_count}.jpg')
            cv2.imwrite(filename, frame)
            print(f"📸 Screenshot saved: {filename}")
    
    # Cleanup
    cap.release()
    cv2.destroyAllWindows()

In [ ]:

def adjust_camera_settings(camera_index=0):  
    print("\n🔧 Testing camera settings...")
    
    cap = cv2.VideoCapture(camera_index)
    
    if not cap.isOpened():
        print("❌ Could not open camera")
        return
    
    # Try different resolutions
    resolutions = [
        (640, 480),    # VGA - Fast
        (1280, 720),   # HD - Good balance
        (1920, 1080),  # Full HD - Slower
    ]
    
    print("\n📊 Available resolutions:")
    for width, height in resolutions:
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
        
        actual_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        actual_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        
        if actual_width == width and actual_height == height:
            print(f"   ✅ {width}x{height}")
        else:
            print(f"   ❌ {width}x{height} (not supported)")
    
    cap.release()

In [ ]:
def checkAvailableCams():
    # Check for available cameras
    print("\n📹 Checking for cameras...")
    available_cameras = []
    
    for i in range(5):  # Check first 5 camera indices
        cap = cv2.VideoCapture(i)
        if cap.isOpened():
            available_cameras.append(i)
            cap.release()
    
    if available_cameras:
        print(f"✅ Found cameras at indices: {available_cameras}")
        print(f"💡 Default camera: {available_cameras[0]}")
    else:
        print("❌ No cameras found!")

In [ ]:
#Test Camera Access
if __name__ == "__main__": 
    checkAvailableCams()
    print("\nStarting camera test in 3 seconds...")
    time.sleep(3)
    
    test_camera_access()